# Tìm hiểu về Định dạng Chat với SmolLM2

Trong notebook này, chúng ta sẽ học về cách sử dụng định dạng Chat với mô hình SmolLM2. Định dạng Chat giúp cấu trúc hóa các tương tác giữa người dùng và mô hình ngôn ngữ, đảm bảo các phản hồi nhất quán và phù hợp với ngữ cảnh.

In [1]:
# Cài đặt các thư viện cần thiết
# !pip install transformers datasets trl huggingface_hub

# Đăng nhập vào Hugging Face
from huggingface_hub import login

login()

# Để thuận tiện, bạn có thể tạo một biến môi trường chứa `token hub` của bạn dưới dạng HF_TOKEN

In [2]:
# Các thư viện cần thiết 
from transformers import AutoModelForCausalLM, AutoTokenizer
from trl import setup_chat_format
import torch

## Định dạng chat của mô hình SmolLM2

Hãy khám phá cách sử dụng *định dạng chat* với mô hình `SmolLM2`. Chúng ta sẽ định nghĩa một cuộc hội thoại đơn giản và áp dụng *định dạng chat*.

In [3]:
# Thiết lập phần cứng một cách linh động
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps" if torch.backends.mps.is_available() else "cpu"
)
print(device)

model_name = "HuggingFaceTB/SmolLM2-135M"
model = AutoModelForCausalLM.from_pretrained(
    pretrained_model_name_or_path=model_name
).to(device)
tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name_or_path=model_name)
model, tokenizer = setup_chat_format(model=model, tokenizer=tokenizer)

cpu


In [4]:
# Ví dụ về tin nhắn
messages = [
    {"role": "user", "content": "Hello, how are you?"},
    {
        "role": "assistant",
        "content": "I'm doing well, thank you! How can I assist you today?",
    },
]

# Áp dụng định dạng chat mà không tokenize

Tokenizer sẽ biểu diễn cuộc hội thoại dưới dạng chuỗi với các *token đặc biệt* để mô tả vai trò của người dùng và trợ lý.


In [5]:
input_text = tokenizer.apply_chat_template(messages, tokenize=False)

print("Cuộc hội thoại với định dạng:", input_text)

Cuộc hội thoại với định dạng: <|im_start|>user
Hello, how are you?<|im_end|>
<|im_start|>assistant
I'm doing well, thank you! How can I assist you today?<|im_end|>



# Giải mã cuộc hội thoại

Lưu ý rằng cuộc hội thoại được biểu diễn như trên nhưng với thêm một tin nhắn trợ lý.


In [6]:
input_text = tokenizer.apply_chat_template(
    messages, tokenize=True, add_generation_prompt=True
)

print("Cuộc hội thoại được giải mã:", tokenizer.decode(token_ids=input_text))

Cuộc hội thoại được giải mã: <|im_start|>user
Hello, how are you?<|im_end|>
<|im_start|>assistant
I'm doing well, thank you! How can I assist you today?<|im_end|>
<|im_start|>assistant



# Tokenize cuộc hội thoại

Tất nhiên, *tokenizer* cũng tokenize cuộc hội thoại và các token đặc biệt thành các id liên quan đến từ vựng của mô hình.



In [7]:
input_text = tokenizer.apply_chat_template(messages, add_generation_prompt=True)

print("Cuộc hội thoại được tokenize:", input_text)

Cuộc hội thoại được tokenize: [1, 4093, 198, 19556, 28, 638, 359, 346, 47, 2, 198, 1, 520, 9531, 198, 57, 5248, 2567, 876, 28, 9984, 346, 17, 1073, 416, 339, 4237, 346, 1834, 47, 2, 198, 1, 520, 9531, 198]


<div style='background-color: lightblue; padding: 10px; border-radius: 5px; margin-bottom: 20px; color:black'>
    <h2 style='margin: 0;color:blue'>Bài tập: Xử lý dataset cho SFT</h2>
    <p>Lấy một dataset từ Hugging Face hub và xử lý nó cho SFT. </p>
    <p><b>Các bài tập</b></p>
    <p>🐢 Chuyển đổi dataset `HuggingFaceTB/smoltalk` sang định dạng <strong>chatml</strong>.</p>
    <p>🐕 Chuyển đổi dataset `openai/gsm8k` sang định dạng <strong>chatml</strong>.</p>
</div>

**Định dạng ChatML:**

```sh
<|im_start|>user
Hi there!<|im_end|>
<|im_start|>assistant
Nice to meet you!<|im_end|>
<|im_start|>user
Can I ask a question?<|im_end|>
<|im_start|>assistant
```

In [8]:
from IPython.core.display import display, HTML

display(
    HTML(
        """<iframe
  src="https://huggingface.co/datasets/HuggingFaceTB/smoltalk/embed/viewer/all/train?row=0"
  frameborder="0"
  width="100%"
  height="360px"
></iframe>
"""
    )
)

/tmp/ipykernel_1767/4068687702.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython.display
  from IPython.core.display import display, HTML


In [9]:
from datasets import load_dataset

ds = load_dataset("HuggingFaceTB/smoltalk", "everyday-conversations")


def process_dataset(sample):
    # TODO: 🐢 Chuyển đổi dữ liệu sang định dạng ChatML
    # Sử dụng `tokenizer.apply_chat_template` để áp dụng định dạng hội thoại
    sample = sample.get('messages', [])
    sample = tokenizer.apply_chat_template(sample, tokenize=True, add_generation_prompt=True)
    return {'text': sample}


ds = ds.map(process_dataset)

Map:   0%|          | 0/2260 [00:00<?, ? examples/s]

Map:   0%|          | 0/119 [00:00<?, ? examples/s]

In [11]:
print("Cuộc hội thoại được giải mã:", tokenizer.decode(token_ids=ds['train']['text'][0]))

Cuộc hội thoại được giải mã: <|im_start|>user
Hi there<|im_end|>
<|im_start|>assistant
Hello! How can I help you today?<|im_end|>
<|im_start|>user
I'm looking for a beach resort for my next vacation. Can you recommend some popular ones?<|im_end|>
<|im_start|>assistant
Some popular beach resorts include Maui in Hawaii, the Maldives, and the Bahamas. They're known for their beautiful beaches and crystal-clear waters.<|im_end|>
<|im_start|>user
That sounds great. Are there any resorts in the Caribbean that are good for families?<|im_end|>
<|im_start|>assistant
Yes, the Turks and Caicos Islands and Barbados are excellent choices for family-friendly resorts in the Caribbean. They offer a range of activities and amenities suitable for all ages.<|im_end|>
<|im_start|>user
Okay, I'll look into those. Thanks for the recommendations!<|im_end|>
<|im_start|>assistant
You're welcome. I hope you find the perfect resort for your vacation.<|im_end|>
<|im_start|>assistant



In [12]:
display(
    HTML(
        """<iframe
  src="https://huggingface.co/datasets/openai/gsm8k/embed/viewer/main/train"
  frameborder="0"
  width="100%"
  height="360px"
></iframe>
"""
    )
)

In [16]:
ds = load_dataset("openai/gsm8k", "main")


def process_dataset(sample):
    # TODO: 🐕 Chuyển đổi dữ liệu sang định dạng ChatML

    # 1. Tạo định dạng tin nhắn với vai trò và nội dung
    format_message = [
        {"role": "user", "content": sample.get('question', '')},
        {"role": "assistant", "content": sample.get('answer', '')},
    ]

    # 2. Áp dụng định dạng chat cho các mẫu bằng tokenizer
    sample = tokenizer.apply_chat_template(format_message, add_generation_prompt=True)
    return {'token': sample}


ds = ds.map(process_dataset)

Map:   0%|          | 0/7473 [00:00<?, ? examples/s]

Map:   0%|          | 0/1319 [00:00<?, ? examples/s]

In [18]:
ds['train']['token']

[[1,
  4093,
  198,
  62,
  6927,
  542,
  3459,
  23026,
  288,
  216,
  36,
  40,
  282,
  874,
  2428,
  281,
  4124,
  28,
  284,
  965,
  1041,
  3459,
  2745,
  347,
  800,
  23026,
  281,
  2405,
  30,
  1073,
  800,
  23026,
  1250,
  36366,
  542,
  5948,
  13587,
  281,
  4124,
  284,
  2405,
  47,
  2,
  198,
  1,
  520,
  9531,
  198,
  62,
  6927,
  542,
  3459,
  216,
  36,
  40,
  31,
  34,
  446,
  22646,
  36,
  40,
  31,
  34,
  45,
  34,
  36,
  7791,
  34,
  36,
  23026,
  281,
  2405,
  30,
  198,
  62,
  6927,
  542,
  3459,
  216,
  36,
  40,
  27,
  34,
  36,
  446,
  22646,
  36,
  40,
  27,
  34,
  36,
  45,
  39,
  34,
  7791,
  39,
  34,
  23026,
  13587,
  281,
  4124,
  284,
  2405,
  30,
  198,
  1229,
  216,
  39,
  34,
  2,
  198,
  1,
  520,
  9531,
  198],
 [1,
  4093,
  198,
  71,
  1059,
  38668,
  1885,
  33,
  34,
  354,
  5353,
  327,
  3383,
  672,
  9584,
  30,
  718,
  15955,
  28,
  1041,
  915,
  1250,
  216,
  37,
  32,
  3487,
  282,
  338

## Kết luận

Trong bài học này, chúng ta đã học được cách áp dụng định dạng chat cho các mô hình khác nhau, cụ thể là `SmolLM2`. Bằng cách cấu trúc các tương tác với định dạng chat, chúng ta có thể đảm bảo rằng các mô hình ngôn ngữ cung cấp các phản hồi nhất quán và phù hợp với ngữ cảnh.

Trong bài tập, bạn đã thử chuyển đổi một dataset sang định dạng *chatml*. May mắn thay, trong thực tế thư viện TRL sẽ tự động làm điều này. Tuy vậy, việc hiểu những gì đang thực sự diễn ra đằng sau các hàm sẽ hữu ích hơn cho bạn.